In [8]:
from bs4 import BeautifulSoup as bsoup
import urllib.robotparser
import requests
from collections import defaultdict
import pandas as pd
from time import sleep

In [9]:
# scrape hop names and links from main page
response = requests.get("https://beermaverick.com/hops")
if response.status_code != 200:
    print("response code not 200")

hops_page = response.text

# print(hop_soup)

In [10]:
hop_soup = bsoup(hops_page, 'html')

In [11]:
hops = hop_soup.find_all('div', {'class': 'box-inner-p-bigger box-single'})
hops = hops[1].find_all('a')
hops.pop(-1)
hop_list = []
for i in hops:
    hop = {}
    hop_link = "https://beermaverick.com" + str(i).split('"')[1]
    hop_name = str(i).split('"')[2].split(">")[1].split("<")[0]
    hop.update({'hop_name':hop_name, 'hop_link':hop_link})
    hop_list.append(hop)
# print(hop_list)

In [5]:
# print(hop_list)

In [12]:
# soup = bsoup(page, 'lxml')
# print(type(soup))
# # print(soup)

NameError: name 'page' is not defined

In [13]:
!curl https://beermaverick.com/robots.txt

User-agent: *
Disallow: /wp-admin/
Allow: /wp-admin/admin-ajax.php


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    67  100    67    0     0    221      0 --:--:-- --:--:-- --:--:--   221


In [14]:
rp = urllib.robotparser.RobotFileParser()
rp.set_url("https://beermaverick.com/hop/robots.txt")
rp.read()
rp.can_fetch("*", ":https://beermaverick.com/hop/mosaic/")

True

In [18]:
hops_all = []
for i in hop_list:
    tables = pd.read_html(i['hop_link'])
    table1 = tables[0].values.tolist()
    for t in table1:
        i.update({t[0]:t[1]})

    response = requests.get(i['hop_link'])  
    if response.status_code != 200:
        print("response code not 200")
    else: 
        page = response.text
        soup = bsoup(page, 'html')
        bv = soup.find_all("table", attrs={"class": "brewvalues"})

    if i == 0:
        # print(i['hop_link'])
        bv_name = bv[0].find_all('th')
        bv_value = bv[0].find_all('td')
        bv_names = []
        bv_names_descrip = []
        
     
        for i in bv_name:
            bv_names.append(i.contents[0].strip('›\xa0 '))
            if str(i.contents[1]) == '<br/>':
                bv_names_descrip.append(str(i.contents[2]).strip('<small class="text-muted">').strip('/<').rsplit('  <a')[0])
            else:
                bv_names_descrip.append(str(i.contents[1]).strip('<small class="text-muted">').strip('/<').rsplit(' <a')[0])
    ta = soup.find_all('em')
    if len(ta) == 0:
        i.update({'tags': None})
    else:
        tags = []
        print(ta[0].text)
        print(ta[0].text.split('#'))
        for tag in ta[0].text:
            print(tag)
            if tag[0] == '#':
                tags.append(tag.text.lstrip('#'))
        i.update({'tags':tags})
    bv_values = []
    for j in bv_value:
        if j.contents[0] == "Total Oil Breakdown:":
            pass 
        else:
            bv_values.append(str(j.contents[0]))

    hops = [(bv_names[i], bv_values[i]) for i in range(len(bv_values))]
    hops_data = {}
    hops_data.update(i)
    for i in hops:
        hops_data.update({i[0]:i[1]})
    print(hops_data)
    hops_all.append(hops_data)
    sleep(8)
    print()



NameError: name 'bv_value' is not defined

In [ ]:
hops_all_df = pd.DataFrame(hops_all)
hops_all_df

NameError: name 'pd' is not defined